# 데이터 전처리

In [2]:
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings(action = 'ignore') # warning 메시지 표시x

C:\Users\mario\AppData\Local\Temp\ipykernel_16948\4048632335.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 0)네이버 쇼핑 리뷰 -> 200,000개 리뷰

In [21]:
naver_review = pd.read_table('../2. crawling_data/naver_shopping.txt', names=['ratings', 'reviews'])

# 결측치 확인 -> x
naver_review.info() 

# 4,5점은 긍정 / 1,2점은 부정으로 라벨링
naver_review['ratings'].value_counts()
'''
5    81177
2    63989
1    36048
4    18786
'''
# 3점은 np.nan -> 라벨링x
naver_review['label'] = np.where(naver_review['ratings'] >=4, 1, np.where(naver_review['ratings'] <= 2, 0, np.nan))
naver_review['label'] = naver_review['label'].astype('Int64')
naver_review['label'].value_counts()
'''
0    100037
1     99963
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ratings  200000 non-null  int64 
 1   reviews  200000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


label
0    100037
1     99963
Name: count, dtype: Int64

### 1)삼성 버즈 -> 7500개 리뷰
- (전처리) -> 6800개

In [40]:
# 0. 크롤링 데이터 읽기
buds1_df = pd.read_excel('../2. crawling_data/buds/삼성전자 갤럭시 버즈 FE.xlsx') # 1500개
buds2_df = pd.read_excel('../2. crawling_data/buds/삼성전자 갤럭시 버즈2.xlsx') # 1500
buds2_pro_df = pd.read_excel('../2. crawling_data/buds/삼성전자 갤럭시 버즈2 프로.xlsx') # 1500
buds_live_df = pd.read_excel('../2. crawling_data/buds/삼성전자 갤럭시버즈 라이브.xlsx') # 1500
buds_plus_df = pd.read_excel('../2. crawling_data/buds/삼성전자 갤럭시버즈 플러스.xlsx') # 1100

# 1. 5개 합치기
samsung_all_buds = pd.concat([buds1_df, buds2_df, buds2_pro_df, buds_live_df, buds_plus_df], ignore_index=True)
samsung_all_buds # 7100개

# 2. 리뷰제목+리뷰내용 합친 컬럼 새로 만들고, 리뷰제목&내용 컬럼 drop
samsung_all_buds['리뷰제목&내용'] = samsung_all_buds['리뷰 제목'] + " " + samsung_all_buds['리뷰 내용']
samsung_all_buds.drop(['리뷰 제목', '리뷰 내용', '맛 만족도'], axis=1, inplace=True)
samsung_all_buds

# 3. 결측치 확인 및 제거 & 평점3점은 제거
    # 리뷰제목&내용  7097 non-nul
samsung_all_buds.info()
samsung_all_buds = samsung_all_buds.dropna() 
samsung_all_buds = samsung_all_buds.query('`구매자 평점` != 3') # samsung_all_buds = samsung_all_buds[samsung_all_buds['구매자 평점'] != 3]
samsung_all_buds.info()

# 4. 평점 분포 확인
samsung_all_buds['구매자 평점'].value_counts()
'''
5    5703
4     838
2     103
1     159
'''

# 5. 라벨링 -> 4점이상은 1, 2점이하는 0 // 3점은 제외
samsung_all_buds['label'] = np.where(samsung_all_buds['구매자 평점'] >=4, 1, np.where(samsung_all_buds['구매자 평점'] <= 2, 0, np.nan))
samsung_all_buds['label'] = samsung_all_buds['label'].astype('Int64')
samsung_all_buds['label'].value_counts()
'''
label
1    6541
0     262
'''

# 6. Final: 엑셀로 저장
samsung_all_buds.to_excel('samsung_all_buds.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   상품명      7100 non-null   object
 1   구매자 이름   7100 non-null   object
 2   구매자 평점   7100 non-null   int64 
 3   리뷰제목&내용  7097 non-null   object
dtypes: int64(1), object(3)
memory usage: 222.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6800 entries, 0 to 7099
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   상품명      6800 non-null   object
 1   구매자 이름   6800 non-null   object
 2   구매자 평점   6800 non-null   int64 
 3   리뷰제목&내용  6800 non-null   object
dtypes: int64(1), object(3)
memory usage: 265.6+ KB


### 2)애플 에어팟 -> 7495개 리뷰
- (전처리) -> 7260개

In [12]:
# 0. 크롤링 데이터 읽기
airpods_df = pd.read_excel('../2. crawling_data/airpods/애플 에어팟.xlsx') # 1500개
airpods2_df = pd.read_excel('../2. crawling_data/airpods/애플 에어팟 2세대.xlsx') # 1495
airpods2_pro_df = pd.read_excel('../2. crawling_data/airpods/애플 에어팟 2세대 프로_2023.xlsx') # 1500
airpods3_2021_df = pd.read_excel('../2. crawling_data/airpods/애플 에어팟 3세대_2021.xlsx') # 1500
airpods3_2022_df = pd.read_excel('../2. crawling_data/airpods/애플 에어팟 3세대_2022.xlsx') # 1500

# 1. 5개 합치기
apple_all_airpods = pd.concat([airpods_df, airpods2_df, airpods2_pro_df, airpods3_2021_df, airpods3_2022_df], ignore_index=True)
apple_all_airpods # 7495개

# 2. 리뷰제목+리뷰내용 합친 컬럼 새로 만들고, 리뷰제목&내용 컬럼 drop
apple_all_airpods['리뷰제목&내용'] = apple_all_airpods['리뷰 제목'] + " " + apple_all_airpods['리뷰 내용']
apple_all_airpods.drop(['리뷰 제목', '리뷰 내용', '맛 만족도'], axis=1, inplace=True)

# 3. 결측치 확인 및 제거 & 평점3점은 제거
    # 리뷰제목&내용  7097 non-nul
apple_all_airpods.info()
apple_all_airpods = apple_all_airpods.dropna() 
apple_all_airpods = apple_all_airpods.query('`구매자 평점` != 3') # apple_all_airpods = apple_all_airpods[apple_all_airpods['구매자 평점'] != 3]

# 4. 평점 분포 확인
apple_all_airpods['구매자 평점'].value_counts()
'''
5    6234
4     716
1     227
2      82
'''

# 5. 라벨링 -> 4점이상은 1, 2점이하는 0 // 3점은 제외
apple_all_airpods
apple_all_airpods['label'] = np.where(apple_all_airpods['구매자 평점'] >=4, 1, np.where(apple_all_airpods['구매자 평점'] <= 2, 0, np.nan))
apple_all_airpods['label'] = apple_all_airpods['label'].astype('Int64')
apple_all_airpods['label'].value_counts()

# 6. Final: 엑셀로 저장
apple_all_airpods.to_excel('apple_all_airpods.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7495 entries, 0 to 7494
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   상품명      7495 non-null   object
 1   구매자 이름   7494 non-null   object
 2   구매자 평점   7495 non-null   int64 
 3   리뷰제목&내용  7495 non-null   object
dtypes: int64(1), object(3)
memory usage: 234.3+ KB


'\nlabel\n1    6541\n0     262\n'

### 3)삼성+애플

In [20]:
samsung_all_buds = pd.read_excel('samsung_all_buds.xlsx') # 1500개
apple_all_airpods = pd.read_excel('apple_all_airpods.xlsx') # 1500개

samsung_all_buds # 6800개
apple_all_airpods # 7259

coupang_reviews = pd.concat([samsung_all_buds, apple_all_airpods], ignore_index=True)
coupang_reviews # 14059

coupang_reviews['label'].value_counts()
'''
label
1    13488
0      571
'''

# 최종 전처리 데이터를 xlsx로 저장
coupang_reviews.to_excel('coupang_reviews.xlsx', index=False)